# Lab 5
## Data Structures & Algorithms

## Today

* [Brief introduction to databases](#databases)
* [Example: a blog page without a database](#blog)
* [Why Use a Database?](#why)
* [Databases in Flask](#databases_flask)
* [Example continued: a blog page (now with database)](#blog_database)
* [Exercises](#exercises)

# What are Databases? <a class="anchor" id="databases"></a>

* Databases are structured collections of data, typically stored and accessed electronically through computer systems.
* Users interact with databases via Database Management Systems (DBMS), which provide tools for efficiently storing, retrieving, and managing data.
* Databases are fundamental to most software applications, enabling organised and scalable data handling.

Databases can roughly be categorised into **two types**: 

1. **Relational Databases(RDBMS)** : 
    * Organise data into tables with rows and columns.
    * Use Structured Query Language (SQL) for data manipulation and retrieval.
    * Ideal for structured data with clear relationships.
2. **NoSQL databases**: 
    * Designed for unstructured or semi-structured data.
    * Utilise diverse data models (e.g., document, key-value, graph, columnar).
    * Employ various query languages tailored to specific models.

We will focus on relational databases!

# Example: a blog post without a database <a class="anchor" id="blog"></a>

Let's extend our existing Flask application, to include a blog page. 
* Since our home page didn't include anything yet, we will turn this into the blog page. 
* Initially, we'll hardcode the blog posts directly into our Flask app.
* Then later we'll improve it by storing blog posts in a database. 

Our code will be following *roughly* [part 4](https://www.youtube.com/watch?v=cYWiDiIUxQc&list=PL-osiE80TeTs4UjLw5MM6OjgkjFeUxCYH&index=4) of the Flask tutorial (to which I linked a couple of weeks ago).

First off, let's make sure we have the necessary set up in place. 

### Lab 2 recap setup (nothing new!)

Before, we add anything new, we'll need the following files:

In the root `my_flask_app` directory:

#### 1. `flaskapp.py` 
```python
from flask import Flask, render_template

app = Flask(__name__)

@app.route("/")
@app.route("/home")
def home():
    return render_template("home.html", heading="A heading variable passed to the template")

@app.route("/hello")
@app.route("/hello/<city>")
def hello(city=None):
    return render_template('hello.html', city=city)

@app.route("/about")
def about():
    return render_template('about.html', title='About page')

if __name__ == '__main__':
    app.run(debug=True)
```

In a `templates` subdirectory:

#### 2. `layout.html`
```html
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>My Web App</title>
    {% block head %}
    {% endblock %}
</head>
<body>
    <header>
        <div id="container">
            <h1 class="logo">My web app</h1>
            <nav>
                <ul class="menu">
                    <li><a href="{{ url_for('home') }}">Home</a></li>
                    <li><a href="{{ url_for('about') }}">About</a></li>
                    <li><a href="{{ url_for('hello') }}">Hello</a></li>
                </ul>
            </nav>
        </div>
    </header>
    
    <div class="container">
        {% block content %}
        {% endblock %}
    </div>
</body>
</html>
```

#### 3. `home.html`
```html
{% extends "layout.html" %}

{% block head %}
    <title>Home</title>
    <meta name="description" content="My home page, on which I showcase my work!">
    <meta name="keywords" content="personal home page">
{% endblock %}

{% block content %}
    <div class="myDiv">
        {% if heading %}
            <h1>{{ heading }}</h1>
        {% else %}
            <h1>Home Page</h1>
        {% endif %}
        <p>This is where you will put the content of this page.</p>
    </div>
{% endblock %}
```

#### 4. `about.html`
```HTML
{% extends "layout.html" %}

{% block head %}
    <title>About</title>
    <meta name="description" content="My home page, where I'll introduce myself!">
    <meta name="keywords" content="personal home page">
{% endblock %}

{% block content %}
    <h1>About Page</h1>
    <p>This is where I'll write something about myself.</p>
{% endblock %}
```

With this set up, we'll be able to begin extending our app with a blog database.

## Step 1: Hardcoding Blog Posts

We'll start by hardcoding blog posts directly into our Flask app.

Add the `posts = []` hardcoding directly into the `flaskapp.py`, as below:

```python
from flask import Flask, render_template

app = Flask(__name__)

# this part is new:
posts = [
    {
        'id': 1,
        'author': 'Your name',
        'title': 'Blog Post 1',
        'content': 'First post content',
        'date_posted': '4 March 2024'
    },
    {
        'id': 2,
        'author': 'Your name',
        'title': 'Blog Post 2',
        'content': 'Second post content',
        'date_posted': '5 March 2024'
    }
]
# route for the home page, which is where the blog posts will be shown, so we need to add 'posts=posts
@app.route("/")
@app.route("/home")
def home():
    return render_template('home.html', posts=posts)


# this is same as before:
# route for the about page
@app.route("/about")
def about():
    return render_template('about.html', title='About page')

if __name__ == '__main__':
    app.run(debug=True)
```

## Step 2: Creating a Blog Template

Now, we'll create a template to display the blog posts.

Add the following code to the `content` block of your `home.html`

```html
{% block content %}
    <div class="myDiv">
        {% if heading %}
            <h1>{{ heading }}</h1>
        {% else %}
            <h1>Home Page</h1>
        {% endif %}
        <p>Here is an early version of my blog.</p>
    </div>

    {% for post in posts %}
    <div class="article">
      <h2>{{ post.title }}</h2>
      <p>By {{ post.author }} on {{ post.date_posted }}</p>
      <p>{{ post.content }}</p>
    </div>
  {% endfor %}
  {% endblock %}
```

## Step 3: Running the Flask App

Finally, we'll run our Flask app to see the blog page by running 
1. Navigate to the project directory in the command line
2. Run the app: `python flaskapp.py`. Remember this will run in debug mode only if you have 
    ```python
    if __name__ == '__main__':
        app.run(debug=True)
    ```
See Lab 2 for other command line options.
- such as `flask --app flaskapp run --debug`

# Why Use a Database in a Web App? <a class="anchor" id="why"></a>

When building a Flask application, using a database instead of hardcoding data or using spreadsheets offers several advantages:

- **Data Integrity**: Databases enforce data integrity through constraints and relationships, ensuring consistent and accurate data (e.g., enforcing correct data types or value limits).
- **Scalability**: Designed to efficiently manage large data volumes and handle multiple user requests, making them ideal for applications with expanding user bases.
- **Querying Flexibility**: Databases support powerful query languages (e.g., SQL) for retrieving, filtering, and analyzing data with complex criteria.
- **Concurrency Control**: Built-in mechanisms manage simultaneous data access, preventing conflicts and maintaining data consistency.

**For example, let's consider a website with a blog:**  
- Consider a blog website. Storing blog posts and related data (e.g., user information) in a database enables efficient retrieval, searching, and filtering of posts by author, category, or date. It also simplifies adding features like comments, likes, and analytics, which would be cumbersome with hardcoded data or spreadsheets.

# Example Database Structure: Blog Posts

For our blog posts situation, a database structure for storing the posts and related data might include the following tables:

1. **Posts Table**:

| post_id | title           | content                           | author_id | date_posted | category_id |
|---------|-----------------|-----------------------------------|-----------|-------------|-------------|
| 1       | First Post      | Content of the first blog post.   | 1         | 04-03-2024  | 2           |
| 2       | Second Post     | Content of the second blog post.  | 2         | 05-03-2024  | 2           |

2. **Authors Table**:

| author_id | name      | email             | bio                            |
|-----------|-----------|-------------------|--------------------------------|
| 1         | John Doe  | john@example.com  | Lorem ipsum dolor sit amet...  |
| 2         | Jane Smith| jane@example.com  | Nulla vitae elit libero...     |

3. **Categories Table**:

| category_id | name     | description            |
|-------------|----------|------------------------|
| 1           | Tech     | In this category, we ..|
| 2           | Lifestyle| The lifestyle catego...|

This database structure allows us to efficiently manage and query blog posts, authors, and categories, facilitating the development of a robust and scalable blog application.

# Querying the Database

Once the database is set up with the above structure, we would normally query it using SQL. For example, to retrieve all blog blog titles along with their authors and categories, we might use the following SQL query:

```sql
SELECT p.title, a.name AS author, c.name AS category
FROM Posts p
JOIN Authors a ON p.author_id = a.author_id
JOIN Categories c ON p.category_id = c.category_id;
```

* the `SELECT` clause specifies the columns to retrieve:
    * `p.title`: The title of the blog post from the `Posts` table (aliased as `p`).
    * `a.name AS author`: The author's name from the `Authors` table (aliased as `a`), renamed as `author` in the output.
    * `c.name AS category`: The category name from the `Categories` table (aliased as `c`), renamed as `category` in the output.
* the `FROM` clause specifies the main table: Posts, which is given the alias p for shorthand reference.
* the `JOIN Authors a ON p.author_id = a.author_id`
    * an inner join connects `Posts` table to `Authors` table using the `author_id` column.
* `JOIN Categories c ON p.category_id = c.category_id`
    * second inner join on `category_id` columns


The resulting table would look like this:

| title       | author     | category   |
|-------------|------------|------------|
| First Post  | John Doe   | Lifestyle  |
| Second Post | Jane Smith | Lifestyle  |

# Databases in Flask <a class="anchor" id="databases_flask"></a>

To integrate a database into your Flask app, you do not need to know SQL - there are Python library that help you work with databases by using Python classes (rather than having to deal with the database tables directly) (a.k.a in a 'Pythonic' way). 

Specifically, to integrate databases into flask apps, we can use a library called `Flask-SQLAlchemy`, based on another popular Python library called `SQLAlchemy`. 

`SQLAlchemy` is an **SQL toolkit and Object-Relational Mapping (ORM) library for Python**, which allows developers to work with relational databases in the above-mentioned 'Pythonic' way. SQLAlchemy supports various types of SQL databases, including SQLite, MySQL, PostgreSQL, and others. 

This last point underlines one of the advantages of Flask: it provides support for integrating various types of databases into web applications and it does not support any single one of them 'natively'. This means that we can choose a database that fits our application: `SQLAlchemy` allows us to switch between databases, without having to change the code in our app or change the way in which we are accessing the database. For example, we might want to start off with an SQLite databae (which does not require a server) and later -- to deploy an application that requires a server -- we could change to MySQL or PostgreSQL. 

Let us extend our Flask application to use SQLAlchemy and SQLite for database operations. First, let's install the necessary packages. In your command line, activate your virtual environment and install Flask's wrapper for the SQLAlchemy package by running 

`conda install Flask-SQLAlchemy`.

*NB: I had an issue with pip here; if so try `conda install flask-sqlalchemy`*

# Example continued: a blog post (now with database) <a class="anchor" id="blog_database"></a>

Now, let's modify our Flask application to use SQLAlchemy and SQLite. 

## Step 1: Creating the database model
Copy and paste the following code into your app module (e.g. `flaskapp.py`):

```python
from flask import Flask, render_template
from flask_sqlalchemy import SQLAlchemy
from datetime import datetime

# app instance
app = Flask(__name__)

# configuring the SQLite database
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///site.db'

# SQLAlchemy database instance
db = SQLAlchemy(app)
app.app_context().push()

#  a model for blog posts ('models' are used to represent tables in your database).
class BlogPost(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    title = db.Column(db.String(100), nullable=False)
    content = db.Column(db.Text, nullable=False)
    author = db.Column(db.String(50), nullable=False)
    date_posted = db.Column(db.DateTime, nullable=False, default=datetime.utcnow)

    def __repr__(self):
        return f"BlogPost('{self.title}', '{self.date_posted}')"

@app.route("/")
@app.route("/home")
def home():
    # Querying all blog posts from the database
    posts = BlogPost.query.all()
    return render_template('home.html', posts=posts)

# same as before
@app.route("/hello")
@app.route("/hello/<city>")
def hello(city=None):
    return render_template('hello.html', city=city)

@app.route("/about")
def about():
    return render_template('about.html', title='About page')

if __name__ == '__main__':
    app.run(debug=True)
```

⚠️ NB: if you try to run the app now you will get an error message! ⚠️

The `BlogPost` class serves as an abstraction of the blog posts stored in the database. In the context of SQLAlchemy, a 'model' represents the structure of a database table and allows us to interact with the data in an object-oriented manner.

By defining the `BlogPost` class, we're essentially mapping the attributes of a blog post (such as title, content, author, and date posted) to columns in the database table. This abstraction simplifies database operations and promotes code readability and maintainability.

## Step 2: Add New Blog Posts to the Database

At this point our app is broken! Flask is expecting to be able to query a database instance `db`, but it dosn't yet exist! 

To create a new blog post and add it to the database, we first need to add the database according to the model we specified in the code of our main app module (as we did just did above).

*(Eventually this would be placed in a different python module, to make your code more tidy, but for now we are building intuition)*. 

Now we need to create a new instance of the `BlogPost` class, set its attributes (e.g., title, content, author, date posted), and then add it to the database session. Finally, the changes are committed to integrate the new blog post in the database.

First, we will manually add a blog post to the database via the command line. *NB: eventually data will be added to databases through user input (rather than on the command line like this)!*

1. First, go to your command line and make sure you have activated the virtual environment you use for running flask apps and that you are currently in the root directory that contains your app module (e.g. `flaskapp.py`). 

2. Type `python` so that you can write the following code to create the database 'skeleton'. It should return `>>>` indicating python input.
    - (*`ctrl + D` to exit.)*

3. Type the following:
    ```python
    from flaskapp import db
    from flaskapp import BlogPost
    db.create_all()
    ```

4. If you now look at your project directory, there is now a folder called `instances` and inside the database with the name `site.db` was created. 

⚠️ Your flask app should be able to run again now, if without any actual posts ⚠️

The structure of the database has been set up, but there are no rows (i.e. no actual content) in the blog post table yet! Let's add a post to the database. 

5. Underneath `db.create_all()`, now write:

    ```python
    new_post = BlogPost(title='New Post', content='This is a new blog post.', author='Your name')
    db.session.add(new_post)
    db.session.commit()
    ```

    We can also look at the entries in our database by 'querying' the tables in the command line. Let's add another blog post and then look at a list of all the blog posts that are currently stored in the database. 

6. In the command line, run:

    ```python
    new_post_2 = BlogPost(title='New Post 2', content='This is another blog post.', author='Your name')
    db.session.add(new_post_2)
    db.session.commit()
    ```
    This adds a second blog post.
    
7. Now let's query the database to see what posts are currently stored in there.

    ```python
    BlogPost.query.all()
    ```

    The last command will return a list of all blog posts that are currently in the database. 
    
In the code above, you can see that the same command is being used in the code that is triggered when a user goes to the home directory on our app! If we now reload our app, we will see that this new blog post is being displayed. 

We can also now make a small change to our `home.html` template, replacing `post.date_posted` by `post.date_posted.strftime('%Y-%m-%d')`, to make sure our `datetime` variable is displayed neatly as a date.)

## Step 3: Create relationships between tables

We might want a better way to store user information (rather than duplicating this information within the row of each blog post). For this, we create a new `User` class, which will represent a table in our database called `user` (the table objects that are represented by the classes are automatically created with the same name as the class, but with all small letters). 

In the following code, there is now a relationship between the `User` and `BlogPost` model, because users are authors of one or multiple posts (one-to-many relationship). 

*NB: we comment out the `author` column from before in the BlogPost model, and add a new column `user_id`.*

```python
# Defining a model for users
class User(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    name = db.Column(db.String(20), nullable=False)
    posts = db.relationship('BlogPost', backref='author', lazy=True) # nb: db.relationship

    def __repr__(self):
        return f"User('{self.name}', '{self.id}')"

# Defining a model for blog posts 
class BlogPost(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    title = db.Column(db.String(100), nullable=False)
    content = db.Column(db.Text, nullable=False)
    # author = db.Column(db.String(50), nullable=False)
    date_posted = db.Column(db.DateTime, nullable=False, default=datetime.utcnow)
    user_id = db.Column(db.Integer, db.ForeignKey('user.id'), nullable=False) # nb: db.ForeignKey

    def __repr__(self):
        return f"BlogPost('{self.title}', '{self.date_posted}')"
```

To establish this relationship, we created a `posts` attribute in the `User` model.

⚠️ NB: this is **not** a db column but a `relationship` to the `BlogPost` model. Under the hood, it performs an SQL query to retrieve all blog posts where the `user_id` matches the user's `id`.

* The `backref=author` parameter in the `User` model automatically adds a virtual author attribute to the `BlogPost` model. This works similarly to having an explicit `author` column but is more efficient and dynamic. (Notice that we’ve removed the hardcoded `author` column from the `BlogPost` model.)
* To explicitly link each blog post to a user, we added a `user_id` column in the `BlogPost` model. This column uses `db.ForeignKey('user.id')` to establish the relationship, enforcing that each post must be associated with a valid user.
* In essence, we've replaced the original `author` column in `BlogPost` with a dedicated `User` model and connected them through:
    * `db.ForeignKey` → A one-way link from `BlogPost` to User that enforces data integrity by ensuring every blog post is linked to a valid user.
    * `backref` → A two-way relationship that allows:
        * Accessing a post's author with post.author.
        * Accessing all of a user’s posts with user.posts.

* The posts attribute on the `User` model dynamically queries the database to fetch all related blog posts. Again, this is not a physical column in the database but a convenient Object-Relational Mapping (ORM) feature that runs SQL in the background.

Let us clear the current database and create a row in the user table. In the command line:

```python
from flaskapp import db
from flaskapp import User, BlogPost
db.drop_all()
db.create_all()
user1 = User(name='Henry Baker')
db.session.add(user1)
db.session.commit()
```

If you reload your app now, nothing should show up, since you have only created a user and no blog post exists yet. You can also see this by running 

```python
user1.posts
```

which should return an empty list. Let us create a post by this user:

```python
user1.id
post1 = BlogPost(title='First post', content='Some text Henry has written.', user_id=user1.id)
post2 = BlogPost(title='Another post', content='Some other content.', user_id=user1.id)
db.session.add(post1)
db.session.add(post2)
db.session.commit()
```

Re-running the `user1.posts` command should show you a list of these two posts. If you reload your app now, it should show the blog posts you have just added. 

NB: we also need to make a small change to the `home.html` template. We need to refer to the author of the post as `{{ post.author.name }}` instead of `{{ post.author }}`, since the latter refers to the entire `User` class, as defined through the `backref` parameter, when we defined the relationship. 

# Exercises <a class="anchor" id="exercises"></a>

### Exercise 1

Go through the steps above for the example WITH the database and get it running as a flask app.

### Exercise 2

Now, add an extra column to the users table that contains a short biography about the user. To do this, you need to update the `User` class accordingly and then go to your command line, open a python shell, drop the current database, and recreate it (like we did above after creating the `User` table in the first place). 

### Solution 2

Your `User` model should look like this:

```python
class User(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    name = db.Column(db.String(20), nullable=False)
    bio = db.Column(db.Text, nullable=False, default='This author did not provide a biography.')
    posts = db.relationship('BlogPost', backref='author', lazy=True)

    def __repr__(self):
        return f"User('{self.name}', '{self.id}')"
```

and you then need to run the following lines in the command line:

```python
from flaskapp import db, User, BlogPost
db.drop_all()
db.create_all()
user = User(name='SpongeBob SquarePants', bio='A sponge that lives on the bottom of the sea.')
db.session.add(user)
post1 = BlogPost(title='First post', content='Some text.', user_id=user.id)
post2 = BlogPost(title='Second post', content='Some more content.', user_id=user.id)
db.session.add(post1)
db.session.add(post2)
db.session.commit()
```

### Exercise 3

Include this short user biography above the content of each blog post by updating your `home.html` template.

### Solution 3

Your `home.html` template could look something like this:

```html
{% extends 'layout.html' %}
{% block content %}
  {% for post in posts %}
    <div class="article">
      <h2>{{ post.title }}</h2>
      <p>By {{ post.author.name }} on {{ post.date_posted.strftime('%Y-%m-%d') }}</p>
      <p>Author biography: {{ post.author.bio }}</p>
      <p>{{ post.content }}</p>
    </div>
  {% endfor %}
{% endblock %}
```

### Exercise 4

To understand in what way the SQLAlchemy statements connect to the SQL queries you already know, you can `stringify` SQLAlchemy statements. Go to your command line and open a python shell by typing `python` and import the database connection and the two models you created, followed by importing the `select` and `text` methods from `sqlalchemy`:

```python
from flaskapp import db, User, BlogPost
from sqlalchemy import select, text
```

Now write the SQLAlchemy equivalent of selecting all columns from the BlogPosts table:

```python
statement = select(BlogPost)
```

To see exactly what SQL query this represents, type the following and see what happens (this is where you 'stringify' the SQLAlchemy):

```python
print(str(statement)) 
```

Have a look at the [documentation](https://docs.sqlalchemy.org/en/20/orm/queryguide/index.html) to see how SQL queries are written with the SQLAlchemy ORM. Use the guide to try the following statements, stringifying the statement each time:

1. select the title and the date posted from the blog post table
2. select the content for the post with id 2 (you'll have to use filtering)
3. write a query to retrieve the title of the blog post and the author's name by joining the `BlogPost` table with the `User` table.

### Solution 4

1. `statement = select(BlogPost.title, BlogPost.date_posted)`
2. `statement = select(BlogPost.content).where(BlogPost.id==2)`
3. `statement = select(BlogPost.title, User.name).join(User)`

### Exercise 5

With the last one of your statements from exercise 4, run the query on your database by running

```python
result = db.session.execute(statement)
result.all()
```

Now, re-do the the result that was printed after that last line, this time by writing the statement directly in SQL. For this, create a new variable called `statement_sql`, to which you assign your SQL query wrapped in the `text` wrapper that you imported from `sqlalchemy`. It should look something like this:

```python
statement_sql = text('this is where your SQL query goes')
```

You can then query your database with this statement exactly like you did just now with the `db.session.execute` statement, only replacing the statement variable. 

### Solution 5

```python
statement = select(BlogPost.title, User.name).join(User)
result = db.session.execute(statement)
result.all()
```

The SQL query should look something like this:

```python
statement_sql = text('select x.title, y.name from blog_post x join user y on x.user_id = y.id')
result = db.session.execute(statement_sql)
result.all()
```

### Exercise 6

In your main flask app module, replace the line `posts=BlogPost.query.all()` by a direct SQL query to the database.

### Solution 6

You can query the database using sqlalchemy as follows (replacing `posts=BlogPost.query.all()`). The commented out solution is the direct SQL alternative to writing it as an sqlalchemy query.

(Note that you must import the `select` method from `sqlalchemy` at the top of your module).

```python
statement = select(BlogPost.title, BlogPost.content, BlogPost.date_posted, User.name).join(User)
# statement = text('select x.title, x.content, x.date_posted, y.name from blog_post x join user y on x.user_id = y.id')
result = db.session.execute(statement)
post_ls = result.all()
posts = [{'title': post[0],
          'content': post[1],
          'date_posted': post[2], 
          'author': post[3]} 
         for post in post_ls]
```

You also have to change `post.author.name` in the `home.html` template to `post.author`.